In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from iso3166 import countries

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [2]:
all_trans = pd.read_excel("data/transcripts.xlsx", index_col=0)
weights=pd.read_excel("data/lda_df_weights_update.xlsx", index_col=0)

In [3]:
all_trans.head(5)

,country,year,transcript
0,ALB,1970,33: May I first convey to our President the co...
1,ARG,1970,177.\t : It is a fortunate coincidence that pr...
2,AUS,1970,100.\t It is a pleasure for me to extend to y...
3,AUT,1970,155.\t May I begin by expressing to Ambassado...
4,BEL,1970,"176. No doubt each of us, before coming up to ..."


In [4]:
weights.head(5)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,year,prevalent,country
0,0.082801,0.432069,0.028909,0.264858,0.003181,0.188182,1970,Topic 2,ALB
1,0.525291,0.457330,0.001100,0.009987,0.006222,0.000070,1970,Topic 1,ARG
2,0.360016,0.454747,0.004544,0.004296,0.137150,0.039247,1970,Topic 2,AUS
3,0.648231,0.312949,0.000106,0.027700,0.006662,0.004351,1970,Topic 1,AUT
4,0.582216,0.397412,0.000110,0.002327,0.007637,0.010299,1970,Topic 1,BEL


In [5]:
df_merged=weights
df_merged.head(5)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,year,prevalent,country
0,0.082801,0.432069,0.028909,0.264858,0.003181,0.188182,1970,Topic 2,ALB
1,0.525291,0.457330,0.001100,0.009987,0.006222,0.000070,1970,Topic 1,ARG
2,0.360016,0.454747,0.004544,0.004296,0.137150,0.039247,1970,Topic 2,AUS
3,0.648231,0.312949,0.000106,0.027700,0.006662,0.004351,1970,Topic 1,AUT
4,0.582216,0.397412,0.000110,0.002327,0.007637,0.010299,1970,Topic 1,BEL


In [10]:
drop_values = all_trans.country.unique()
topic_list={"International Diplomacy":"Topic 1","War and Power":"Topic 2","Conflict in Africa":"Topic 3","Middle Eastern Terrorism":"Topic 4","Climate Change":"Topic 5","Communism":"Topic 6"}

In [11]:
# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    
    html.Div([
        html.H1("Modern Data Analytics Project"),
        html.H2("Politicians and Climate Change"),
        html.Div("This Dashboard displays and summarizes some UN-speeches"),
        
        html.Br(),
        
        dcc.Dropdown(
            id='my_dropdown',
            options=[
                {'label': i, 'value': i} for i in drop_values
            ],
            value = 'ALB',
            placeholder='Select a country',
            style={'width': '40%'}
        ),
        
        html.Br(),
        
        html.Div([
        #dcc.Input(id='term_select', type='text', debounce=False,
        #         value='climate', required=True),
        dcc.Dropdown(
            id='topic_select',
            options=[
                {'label': i, 'value': i} for i in topic_list.keys()
            ],
            value = 'Climate Change',
            placeholder='Choose a Topic',
            style={'width': '40%'}
        ),
    ]),
    
    ]),

    html.Div(id='text_out', children=[]),
    
    html.Div([
        dcc.Graph(id='the_graph', figure={})
    ]),

])

@app.callback(
    [Output(component_id='text_out', component_property='children')],
     Output(component_id='the_graph', component_property='figure'),     
    Input(component_id='my_dropdown', component_property='value'),
    Input(component_id='topic_select', component_property='value')
)

def update_graph(my_dropdown, topic_select):
    
    dff = df_merged
    try:
        my_dropdown
    except NameError:
        my_dropdown="ALL"
    
    if my_dropdown == 'ALL':
        data_filtered = dff
        
    else:
        data_filtered = dff.loc[dff['country'] == my_dropdown]
    
    year = np.array(data_filtered["year"].unique())
    weights=np.array(data_filtered[topic_list[topic_select]])

    
    scatter=px.scatter(
        x=year,
        y=weights,
        trendline="lowess",
        ).update_xaxes(range=[min(df_merged["year"]),max(df_merged["year"])])
    
    container = 'Number of speeches containing the term "{}" given by {}'.format(topic_select, my_dropdown)
    
    return container, scatter

In [13]:
# Run app and display result inline in the notebook
app.run_server(mode='external',port=8050)

Dash app running on http://127.0.0.1:8050/


In [12]:
app._terminate_server_for_port("localhost", 8050)

C:\Users\kebro\.conda\envs\jupyterlab\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

